In [ ]:
knitr::opts_chunk$set(echo = TRUE, warning=FALSE, message=FALSE, root.dir = '~/')
getwd()

## Overview

This workflow uses the `dynamAedes` R package to simulate the introduction and population dynamics of the mosquito *Aedes aegypti* across Cyprus in 2024. The temperature data is extracted from the CERRA climate reanalysis and drives the biological rates in the model. Outputs include estimates of daily abundance and spatial spread.

---

## 1. Load Required Packages

In [ ]:
# devtools::install_github("mattmar/dynamAedes@development")
library(terra)
library(tidyterra)
library(dynamAedes)
library(lubridate)
library(dplyr)
library(tidyr)
library(ggplot2)

## 2. Load and Preprocess Temperature Data

This section loads the daily 2m temperature raster data for 2024, reprojects it to its correct coordinate system (Lambert Conformal Conic), and crops it to the geographical extent of Cyprus to reduce computation time.

In [ ]:
myT <- rast("/data/t2m_CERRA_2024.nc")

crs_lcc <- "+proj=lcc +lat_1=50 +lat_2=50 +lat_0=50 +lon_0=8 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs"
crs(myT) <- crs_lcc
names(myT) <- seq.Date(as.Date("2024-01-01"), by="day", length.out = nlyr(myT))

bbox <- ext(930, 980, 280, 330)
myT <- crop(myT, bbox)

---

### 2.1 Apply a Sea Mask

Temperatures over the sea are masked to simulate mosquito dynamics only on land.

In [ ]:
seaMask <- myT[[1]]
seaMask[seaMask > 15] <- NA
seaMask[!is.na(seaMask)] <- 1
myT <- myT * seaMask

plot(myT[[1]], main = "Masked Land Temperature (Day 1)")

---

## 3. Explore Temperature-Dependent Life History Traits

Here we explore how temperature affects mosquito life-history parameters using `dynamAedes::get_rate`.

In [ ]:
ex.temp <- seq(-5, 40, by = 1)
my.r <- get_rate(temp = ex.temp, species = "aegypti", rate = "a.surv")

plot(ex.temp, my.r, type = "l", 
     main = "Adult Survival Rate vs Temperature (Aedes aegypti)",
     xlab = "Temperature (°C)", ylab = "Survival Rate")

Explore spatial variation in survival rates over a few selected days:

In [ ]:
my.r <- get_rate(temp = myT[[c(10, 100, 200, 300, 350)]], species = "aegypti", rate = "a.surv")
plot(my.r)

---

## 4. Prepare Data for Simulation

Temperature data is converted to a format compatible with the `dynamAedes.m()` model: a matrix of temperatures and corresponding cell coordinates.

In [ ]:
df_temp <- as.data.frame(myT, xy = TRUE)
cc <- df_temp[, c("x", "y")]
w <- sapply(df_temp[, -c(1:2)], function(x) as.integer(x * 1000))

### Define Simulation Parameters

In [ ]:
## Define the day of introduction (January 1st is day 1)
str <- "2024-02-01"
## Define the end-day of life cycle (December 31st is the last day)
endr <- "2024-12-31"
## Define the number of eggs to be introduced
ie <- 10000
## Define the number of model iterations
it <- 10 # The higher the number of simulations the better
## Define the number of liters for the larval density-dependent mortality
habitat_liters <- 1000
## Define the number of parallel processes (for sequential iterations set nc=1)
cl <- 10
#subset w to get the first column of the matrix matching the day of introduction 
w <- w[, as.numeric(format(as.Date(str), "%j")):ncol(w)]

---

## 5. Run the Aedes aegypti Model

In [ ]:
simout <- dynamAedes.m(
  species = "aegypti",
  scale = "rg",
  jhwv = habitat_liters,
  temps.matrix = w,
  cells.coords = as.matrix(cc),
  startd = str,
  endd = endr,
  n.clusters = cl,
  iter = it,
  intro.eggs = ie,
  compressed.output = FALSE,
  seeding = TRUE,
  verbose = FALSE
)

outname <- paste0("aegy_syms_uncompressed_workshopNicosia_it", it, "_", Sys.Date(), ".RDS")
saveRDS(simout, outname)

---

## 6. Postprocessing

In [ ]:
simout <- readRDS("aegy_syms_uncompressed_workshopNicosia_it10_2025-09-09.RDS")
dd <- max(simout)

---

### 6.1 PSI Maps

The PSI (Percentage of Successful Introductions) represents the proportion of simulations in which the mosquito population persisted in each cell.

In [ ]:
psi.out <- psi_sp(input_sim = simout, eval_date = c(50, 100, 200, 300), n.clusters = 10)
plot(psi.out)

---

### 6.2 Time Series of Abundance

In [ ]:
breaks <- c(0.25, 0.50, 0.75)
ed <- 1:dd

outdf <- rbind(
  adci(simout, eval_date = ed, breaks = breaks, stage = "Eggs", type = "O"),
  adci(simout, eval_date = ed, breaks = breaks, stage = "Juvenile", type = "O"),
  adci(simout, eval_date = ed, breaks = breaks, stage = "Adults", type = "O")
)

outdf$stage <- factor(outdf$stage, levels = c('Egg', 'Juvenile', 'Adult'))
outdf$Date <- rep(seq.Date(as.Date(str), as.Date(endr) - 2, by = "day"), 3)

ggplot(outdf, aes(x = Date, y = X50., group = stage, color = stage)) +
  ggtitle("Ae. aegypti Interquartile Range of Abundance") +
  geom_ribbon(aes(ymin = X25., ymax = X75., fill = stage), alpha = 0.2, color = NA) +
  geom_line(linewidth = 0.8) +
  labs(x = "Date", y = "Abundance", color = "Stage", fill = "Stage") +
  facet_wrap(~stage, scales = "free_y") +
  theme_light() +
  theme(legend.position = "bottom", text = element_text(size = 16))


#log10 transformed for cleared visualisation
ggplot(outdf, aes(x = Date, y = log10(X50.+1), group = stage, color = stage)) +
  ggtitle("Ae. aegypti Interquartile Range of Abundance") +
  geom_ribbon(aes(ymin = log10(X25.+1), ymax = log10(X75.+1), fill = stage), alpha = 0.2, color = NA) +
  geom_line(linewidth = 0.8) +
  ylim(0, 10)+
  labs(x = "Date", y = "Abundance", color = "Stage", fill = "Stage") +
  facet_wrap(~stage, scales = "free_y") +
  theme_light() +
  theme(legend.position = "bottom", text = element_text(size = 16))


---

## 7. Spatial Summary of Female Adults

In [ ]:
ad.r <- adci(simout, eval_date = c(50, 100, 200, 300), breaks = 0.5,
             stage = "Adults", type = "N", n.clusters = 10)
ad.r <- ad.r$q_0.5

ggplot() +
  geom_spatraster(data = ad.r) +
  scale_fill_viridis_b(option = "viridis", direction = 1,
                       limits = c(0, 500000),
                       breaks = c(0, 50, 100, 1000, 10000, 25000, 50000),
                       oob = scales::squish, na.value = "transparent") +
  labs(x = "Longitude", y = "Latitude", fill = "Daily number of female adults") +
  facet_wrap(~lyr, ncol = 4) +
  theme_classic() +
  theme(legend.position = "bottom",
        text = element_text(size = 12),
        legend.text = element_text(size = 10),
        legend.key.size = unit(2, 'cm')) +
  coord_equal()

---